In [1]:
import os

openai_api_key = os.environ.get("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("The environment variable OPENAI_API_KEY is not set. Please make sure to set it.")

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [5]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.9 MB/s eta 0:00:00


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode


In [7]:
@tool
def imagenow_tool(invoice_id: str):
    """Check payment status in Imagenow."""
    return {"status": "Paid" if invoice_id == "INV-0001" else "Not Paid"}

@tool
def lawson_tool(invoice_id: str):
    """Fetch payment details or exception status from Lawson."""
    return {"payment_method": "Wire Transfer", "payment_date": "2025-01-10", "exception_status": "None"}

@tool
def ivalua_tool(po_number: str):
    """Check transmission status in Ivalua."""
    return {"transmission_status": "Transmitted", "exception_status": "None"}

@tool
def email_tool(recipient: str, message: str):
    """Send an email to the vendor or team."""
    return {"email_status": "Sent"}

@tool
def update_notes_tool(system: str, details: str):
    """Update notes in Imagenow or Ivalua."""
    return {"notes_status": "Updated"}


In [8]:
# Add tools to the ToolNode
tools = [imagenow_tool, lawson_tool, ivalua_tool, email_tool, update_notes_tool]
tool_node = ToolNode(tools)


In [9]:
def call_model(state: MessagesState):
    messages = state["messages"]

    # Workflow steps provided as part of the LLM prompt
    prompt = f"""
    You are an AI orchestrator for the Payment Inquiry workflow. Based on the current state, decide:
    - Which tool to call next.
    - The inputs required for the tool.

    Workflow Steps:
    - Check payment status in Imagenow.
    - If Paid:
      - Fetch payment details from Lawson.
      - Respond to the vendor with payment details.
      - Update notes in Imagenow.
    - If Not Paid:
      - Check PO type (11-digit or 10-digit).
      - For 11-digit PO:
        - Check transmission status in Ivalua.
        - Notify the appropriate person if not transmitted.
        - Check exceptions in Lawson if transmitted.
      - For 10-digit PO:
        - Check exceptions in Lawson.
        - Notify the appropriate person.

    Current State: {state}

    Respond with a tool call in the following format:
    {{
        "tool_name": "ToolName",
        "tool_inputs": {{"key1": "value1", "key2": "value2"}}
    }}
    """

    # LLM response
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    response = llm.invoke(messages + [("system", prompt)])
    print(f"LLM Decision: {response.content}")

    # Simulate parsing the tool call
    tool_call = eval(response.content)  # Use safer parsing in production
    state["tool_calls"] = [tool_call]
    return {"messages": messages + [response]}


In [22]:
def tools_node(state: MessagesState):
    # Process the tool call
    tool_call = state["tool_calls"][0]  # Retrieve the first tool call
    tool_name = tool_call["name"]
    tool_inputs = tool_call["args"]

    # Execute the tool
    result = tool_node.invoke({"messages": [tool_call]})

    # Log the tool execution
    print(f"Tool '{tool_name}' executed with inputs: {tool_inputs}")
    print(f"Tool result: {result}")

    # Add the tool result back to the state
    tool_response = result["messages"][-1]
    state["tool_result"] = tool_response.content
    state["messages"].append(tool_response)

    # Return the updated state
    return state


In [11]:
def should_continue(state: MessagesState):
    # Check if there are more decisions or if workflow should end
    last_message = state["messages"][-1]
    if last_message.tool_calls:  # If there are pending tool calls
        return "tools"
    if "tool_result" in state:  # If there's a tool result, continue to the LLM
        return "agent"
    return END


In [12]:
# Initialize the StateGraph
workflow = StateGraph(MessagesState)

# Add nodes to the workflow
workflow.add_node("agent", call_model)  # LLM decision-making
workflow.add_node("tools", tools_node)  # Tool execution

# Connect nodes
workflow.add_edge(START, "agent")  # Start with the LLM
workflow.add_conditional_edges("agent", should_continue, ["tools", END])  # Decide to continue or stop
workflow.add_edge("tools", "agent")  # Loop back after tools

# Compile the workflow
app = workflow.compile()


In [23]:
# Define the input message
initial_messages = [("human", "Check the payment status of invoice INV-0002.")]

# Execute the workflow
for chunk in app.stream({"messages": initial_messages}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Check the payment status of invoice INV-0002.
LLM Decision: {
    "tool_name": "Imagenow",
    "tool_inputs": {"invoice_id": "INV-0002"}
}
================================== Ai Message ==================================

{
    "tool_name": "Imagenow",
    "tool_inputs": {"invoice_id": "INV-0002"}
}
